In [1]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [2]:
def get_all_variables_from_top_scope(tf, scope):
    #scope is a top scope here, otherwise change startswith part
    return [v for v in tf.all_variables() if v.name.startswith(scope.name)]

def create_model(tf, param1, param2):
    w = tf.get_variable('W', shape=[1], initializer=tf.constant_initializer(param1))
    b = tf.get_variable('b', shape=[1], initializer=tf.constant_initializer(param2))
    y = tf.mul(w, b, name='mul_op')#no need to save this
    return y

In [3]:
import tensorflow as tf
import melt

OLD_SCOPE_NAME = 'scope1'
NEW_SCOPE_NAME = 'scope2'

sess = tf.Session()
m = {}
with tf.variable_scope(NEW_SCOPE_NAME, reuse=None) as topscope:
#with tf.variable_scope(OLD_SCOPE_NAME) as topscope:
    model = create_model(tf, 6.0, 7.0)
    sc_vars = get_all_variables_from_top_scope(tf, topscope)

    print([v.name for v in sc_vars])

    #well here is also a must if you want to laod anoter one
    tf.train.Saver(sc_vars).restore(sess, '/tmp/%s'%NEW_SCOPE_NAME)
    #saver = tf.train.Saver(sc_vars)
    saver = tf.train.Saver()
    print('/tmp/%s'%NEW_SCOPE_NAME)
    saver.restore(sess, '/tmp/%s'%NEW_SCOPE_NAME)
    print([v.name for v in tf.all_variables()])
    print(sess.run(model))
    m['model1'] = model

with tf.variable_scope(OLD_SCOPE_NAME, reuse=None) as topscope:
    model = create_model(tf, 6.0, 7.0)
    sc_vars = get_all_variables_from_top_scope(tf, topscope)

    print([v.name for v in sc_vars])

    #well here is also a must if you want to laod anoter one
    tf.train.Saver(sc_vars).restore(sess, '/tmp/%s'%OLD_SCOPE_NAME)
    print('/tmp/%s'%OLD_SCOPE_NAME)
    print([v.name for v in tf.all_variables()])
    print(sess.run(model))
    m['model2'] = model
    
print(sess.run(m['model1']))
print(sess.run(m['model2']))

[u'scope2/W:0', u'scope2/b:0']
/tmp/scope2
[u'scope2/W:0', u'scope2/b:0']
[ 15.]
[u'scope1/W:0', u'scope1/b:0']
/tmp/scope1
[u'scope2/W:0', u'scope2/b:0', u'scope1/W:0', u'scope1/b:0']
[ 72.]
[ 15.]
[ 72.]
